In [1]:
import os
import sys
from pprint import pprint
import json
import logging
import pandas as pd
import threading
import time

In [9]:
from memoiz import Cache

cache = Cache()


class Greeter:

    def __init__(self):
        self.adv = "Very"

    @cache #User the `cache` decorator in order to add memoization to the `greet` method.
    def greet(self, adj: str) -> str:
        return f"Hello, {self.adv} {adj} World!"


greeter = Greeter()

print("1:", cache._cache)

greeting = greeter.greet("Happy")

print("2:", greeting)

greeting = greeter.greet("Cautious")

print("3:", greeting)

# The cache has memoized the method call using the "Happy" argument and the
# method call using the "Cautious" argument.
print("4:", cache._cache)

# Invalidate the call to `greeter.greet` with the "Happy" argument.
#                               ⮶ instance
cache.invalidate(greeter.greet, greeter, "Happy")
#                         ⮴ method       ⮴ args

print("5:", cache._cache)

# Invalidate the call to `greeter.greet` with the `Cautious` argument.
cache.invalidate(greeter.greet, greeter, "Cautious")

# The cache is empty.
print("6:", cache._cache)

1: {}
2: Hello, Very Happy World!
3: Hello, Very Cautious World!
4: {<bound method Greeter.greet of <__main__.Greeter object at 0x7fa64430e3e0>>: {((<__main__.Greeter object at 0x7fa64430e3e0>, 'Happy'), ()): 'Hello, Very Happy World!', ((<__main__.Greeter object at 0x7fa64430e3e0>, 'Cautious'), ()): 'Hello, Very Cautious World!'}}
5: {<bound method Greeter.greet of <__main__.Greeter object at 0x7fa64430e3e0>>: {((<__main__.Greeter object at 0x7fa64430e3e0>, 'Cautious'), ()): 'Hello, Very Cautious World!'}}
6: {}


In [8]:
from memoiz import Cache

cache = Cache()


@cache
def greet(adj: str) -> str:
    return f"Hello, {adj} World!"


print("1:", cache._cache)

greeting = greet("Happy")

print("2:", greeting)

greeting = greet("Cautious")

print("3:", greeting)

print("4:", cache._cache)

#                       ⮶ args
cache.invalidate(greet, "Happy")
#                ⮴ function

# The call using the "Happy" argument is deleted; however, the call using the "Cautious" is still present.
print("5:", cache._cache)

#                       ⮶ args
cache.invalidate(greet, "Cautious")
#                ⮴ function

# The cache is now empty.
print("6:", cache._cache)

1: {}
2: Hello, Happy World!
3: Hello, Cautious World!
4: {<function greet at 0x7fa6447ee8c0>: {(('Happy',), ()): 'Hello, Happy World!', (('Cautious',), ()): 'Hello, Cautious World!'}}
5: {<function greet at 0x7fa6447ee8c0>: {(('Cautious',), ()): 'Hello, Cautious World!'}}
6: {}


In [ ]:
from memoiz import Cache


cache = Cache()

@cache
def greet(adj: str) -> str:
    return f"Hello, {adj} World!"

def thread_function(args):

    greeting = greet('Happy')

    time.sleep(2)

    print(greeting)
    
t = threading.Thread(target=thread_function, args=(1,))

t.start()
